<a href="https://colab.research.google.com/github/luferIPCA/MIA-MLA-24-25/blob/main/2_Data_Manipulation_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masters' in Applied Artificial Intelligence
## Machine Learning Algorithms Course MLA)

Notebooks for MLA course

by [*lufer*](mailto:lufer@ipca.pt)

(2024)

---

# Part II - Datasets Manipulation (II)

This is the second notebook for datasets manipulation. Cleaning, Normalizing, Initializing are some of the required tasks during dataset preparation for training.

**Contents**:

1. **Data Binning**
2. **Categorical to Numerical**
3. **Dataset Manipulation**



## Environment preparation


### Importing necessary Libraries

In [ ]:
import pandas as pd
import numpy as np

Mounting Drive

In [ ]:

from google.colab import drive

# it will ask for your google drive credentiaals
drive.mount('/content/gDrive/', force_remount=True)

*Loading dataset*

In [ ]:
path="/content/gDrive/MyDrive/Colab Notebooks/MIA - ML - 2024-2025/Datasets/"

In [ ]:
dataset = pd.read_csv(path+'credit_simple.csv', sep=';')
dataset.shape

In [ ]:
dataset



---



##1 - Data Binning

 Grouping technique in data analysis that enables the transformation of continuous data into discrete intervals, providing a clearer picture of the underlying trends and distributions.

 1. Reduce complexity of columns values
 2. Merging columns
 3. Derivating columns
 4. Reduces Noise
 5. Facilitates Data Management:
 6. Handling Missing Data
 7. Eases Categorical Analysis
 8. Enhances Data Visualization
 9. Control Outliers
 10. Others


 Possibe disavantges:

 1. Loss of information
 2. Inconsistency Across Different Datasets
 3. Risk of Overfitting
 4. Others


 See:
 * [Binning](https://medium.com/@tubelwj/pandas-data-preprocessing-data-binning-40614ab13b54)
 * [What is Data Binning](https://www.deepchecks.com/glossary/data-binning/)

In [ ]:
dataset.dtypes



---



There are many categorical (objet) variables

**Preparing the dataset, simplifying it:**

* Identify the dependent variable
* Isolate the feature "CLASSE"

In [ ]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1] #all raws, all remain columns (from 0 to n-1)

Analyse the distribution of variable "PROPOSITO"

In [ ]:
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

The features "Eletrodomesticos" and "qualificação" are poor represented. We can move their values for other feature. For instance, move all their values to feature "outros".

In [ ]:
X.loc[X['PROPOSITO']=='Eletrodomésticos','PROPOSITO'] = 'outros'
#or
#X['PROPOSITO'] = X['PROPOSITO'].replace('Eletrodomésticos', 'outros')
X.loc[X['PROPOSITO']=='qualificação','PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

The dataset has now less columns!

In [ ]:
X.dtypes



---



Lets analyse the variable "IDADE"

In [ ]:
X.groupby(['IDADE']).size()

Several small representations. Lets try to group values into age bins.

In [ ]:
#create a datafrane of all IDADE values
sIdade = X['IDADE']
dfIdade = pd.DataFrame(sIdade)
dfIdade

In [ ]:
dfIdade.groupby('IDADE').size()


Lets create coluns correspondents to bins of 10 years

In [ ]:
# Define bins for grouping ages into periods of 10 years
bins = range(0, 101, 10)  # 0-9, 10-19, 20-29, ..., 90-99, 100+ #array

# Create a new column for the age group (bin)
#fucntion cut() split original dataset
dfIdade['IDADE Group'] = pd.cut(dfIdade['IDADE'], bins)
dfIdade


In [ ]:
#see the new distribution
dfIdade.groupby(['IDADE Group']).size()



---





---



##2 - Categorical to Numeric


In this dataset DATA is a text object (categorical). Lets try to explore it.

 We can parse it to create new columns (Day, Month, etc...)

1. Convert into date type
2. Get parts of it for new columns

In [ ]:
X['DATA']

In [ ]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [ ]:
X['DATA']

In [ ]:
X.dtypes

In [ ]:
#create new columns ANO, MES and DIASEMANA
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [ ]:
#Categorical
X['DIASEMANA']

In [ ]:
X.groupby(['DIASEMANA']).size()

In [ ]:
X.head()

Now we have more (categorical) columns. It is relevant to understand and to convert them in Numerical.

**Why:**
>A structured dataset typically includes numerical and categorical variables. Machine learning algorithms can only process numerical data, not text.

**Approachs:**

1. Replacing categorical values
2. Label Encoding
3. Dummy Columns
4. One-Hot-Encoding

###Replacing Categorical Values

**See:** section 2 on [2_Data_Manipulation_I.ipynb](https://colab.research.google.com/drive/16AMqXZnOu_e8i5EkpSG2GCGQ6EsQbPEo#scrollTo=d3iNajqoK6cA)

###Label Encoding

Technique that is used to convert categorical columns into numerical ones so that they can be fitted by machine learning models.Models only take numerical data.

Convert text values (categorical) in numeric

**Example 1:**

In [ ]:
X['ESTADOCIVIL']

In [ ]:
X['ESTADOCIVIL'].unique()

ESTADOCIVIL has four diferent values labels. The encoding goes from 0 to 4.

In [ ]:
X['PROPOSITO'].unique()

PROPOSITO has eight diferent values labels. The encoding goes from 0 to 8.

In [ ]:
X['DIASEMANA'].unique()

DIASEMANA has seven diferent values labels. The encoding goes from 0 to 7.

In [ ]:
# Import label encoder
from sklearn import preprocessing

labelencoder1 = preprocessing.LabelEncoder()

X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

In [ ]:
X.head()

**ATENTION:**

Label encoding converts the categorical data into numerical ones, but it assigns a unique number(starting from 0) to each class of data. This may lead to the generation of priority issues during model training of data sets. A label with a high value may be considered to have high priority than a label having a lower value.

**Example 2**


In [ ]:
#recover original df

data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 31, 25, None, 27],
        'Gender': ['F', 'M', None, 'M', 'F'],
        'Country':['Portugal','Spain','England','Portugal','Germany'],
        'Salary': [50000, None, 30000, 40000, 60000]}

df = pd.DataFrame(data)

In [ ]:
df


(Try to imprve the datasset: handling the NaN values)

In [ ]:
#Checking the data types of all columns

df.info()

We have three categorical variables (Name, Country, Gender) and two numeric (Salary, Age).

Lets convert Country into a number.

The possible values for Country are: Portugal, Spain, England, Germany. Thus, we can define number for each one, like

* Portugal -> 2
* Spain -> 3
* England -> 0
* Germany -> 1

We can do it manually, but we can use transformers.

In [ ]:
# Import label encoder
from sklearn import preprocessing

# Create a label encoder object
label_encoder = preprocessing.LabelEncoder()

# Encode labels in the 'Country' column
df['Country'] = label_encoder.fit_transform(df['Country'])
print(df.head())

**Attention:**

Label Encoding imposes an arbitrary order on categorical data, which can be misleading.

For mitigate this, the *OneHotEncoding* approach can be used.

###One-Hot Encoding

* One-Hot Encoding creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature. One-Hot Encoding is the process of creating dummy variables.

* It represents categorical variables as binary vectors, creating a new column for each category. Instead of assigning a single numerical label, it ensures that the encoding **does not impose** an ordinal relationship between categories.*

**Example 1:**

In [ ]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1] #all raws, allcolumns from 0 to n-1
X

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Apply OneHotEncoding to the 'Gender' and 'Country' columns
columns_to_encode = ['ESTADOCIVIL']
onehot_encoded = onehot_encoder.fit_transform(X[columns_to_encode])

# Create a new DataFrame with the one-hot encoded columns
encoded_columns = onehot_encoder.get_feature_names_out(columns_to_encode)
encoded_df = pd.DataFrame(onehot_encoded, columns=encoded_columns)

# Concatenate the original DataFrame with the new one-hot encoded columns
newdf = pd.concat([X, encoded_df], axis=1)

# Drop the original 'Gender' and 'Country' columns if no longer needed
newdf.drop(columns=columns_to_encode, inplace=True)

# Show the updated DataFrame
newdf.head()

**Example 2:**

In [ ]:
#recover original df

data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 31, 25, None, 27],
        'Gender': ['F', 'M', None, 'M', 'F'],
        'Country':['Portugal','Spain','England','Portugal','Germany'],
        'Salary': [50000, None, 30000, 40000, 60000]}

df = pd.DataFrame(data)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Apply OneHotEncoding to the 'Gender' and 'Country' columns
columns_to_encode = ['Gender', 'Country']
onehot_encoded = onehot_encoder.fit_transform(df[columns_to_encode])

# Create a new DataFrame with the one-hot encoded columns
encoded_columns = onehot_encoder.get_feature_names_out(columns_to_encode)
encoded_df = pd.DataFrame(onehot_encoded, columns=encoded_columns)

# Concatenate the original DataFrame with the new one-hot encoded columns
df = pd.concat([df, encoded_df], axis=1)

# Drop the original 'Gender' and 'Country' columns if no longer needed
df.drop(columns=columns_to_encode, inplace=True)

# Show the updated DataFrame
print(df)

In [ ]:
df

### Dummy Columns

Create dummy columns from the existing one using *get_dummies*

In [ ]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

In [ ]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix = 'OUTROS')

In [ ]:
z

In [ ]:
X

Concat both dataframes

In [ ]:
newdf = pd.concat([X,z], axis='columns')
newdf.head()

Delete original columns

In [ ]:
newdf.columns

In [ ]:
newdf=newdf.drop(['OUTROSPLANOSPGTO'], axis='columns')
newdf.head()

**Another example**

In [ ]:
dataset = pd.read_csv(filePath+'car_sales.csv', sep=';')

In [ ]:
dataset.head()
#type(dataset)

In [ ]:
import pandas as pd

# Example dataset (as a dictionary for this purpose)
data = {
    'Make': ['Toyota', 'Honda', 'Toyota', 'BMW', 'Nissan', 'Toyota', 'Honda', 'Honda', 'Toyota', ''],
    'Colour': ['White', 'Red', 'Blue', 'Black', 'White', 'Green', '', 'Blue', 'White', 'White'],
    'Odometer': [150043, 87899, None, 11179, 213095, None, None, None, 60000, 31600],
    'Doors': [4, 4, 3, 5, 4, 4, 4, 4, None, 4],
    'Price': ['$4,000', '$5,000', '$7,000', '$22,000', '$3,500', '$4,500', '$7,500', '', '', '$9,700']
}

df = pd.DataFrame(data)

# Create dummy columns for 'Make' using pd.get_dummies
df_dummies = pd.get_dummies(df['Make'], prefix='Make')

# Join the dummy columns back to the original dataframe
df = pd.concat([df, df_dummies], axis=1)

# Display the updated dataframe
print(df)

In [ ]:
#format output like a table
from google.colab import data_table
data_table.enable_dataframe_formatter()
#disabling
#data_table.disable_dataframe_formatter()

In [ ]:
# Set pandas display options to suppress scientific notation
pd.options.display.float_format = '{:.2f}'.format
#reseting
#pd.reset_option('display.float_format')

Later on next pupyte notebboks, other methods for ctegorical to numerical convertion wil ne explored

##3 - Datasets Manipulation (more...)

### Spliting Datasets

***Slicing Data (essential)***

In [ ]:
a = [7, 2, 3, 7, 5, 6, 0, 1]

the instruction *[start:stop]* includes the value in the position *start*, but not in the position *stop*. Both are optional.

Part of the data

In [ ]:
a[1:5]

In [ ]:
a[:5]

In [ ]:
a[3:]

Start from the end of the data

In [ ]:
a[-4:]

In [ ]:
a[-6:-2]

*Values substituition*

In [ ]:
a[3:4] = [6, 3]
a

*Change the step using [start:stop:step]*

In [ ]:
a[::2]

*Special case: Inverting a sequence*

In [ ]:
a[::-1]



---




**Considering existing Datasets**

*Spliting the Dataset by Row*

In [ ]:
len(nba)

*Splitting  Dataframe by groups*

Group the data by column value *year_id*. The newly formed dataframe consists of grouped data with *year_id* = 1947.

In [ ]:
nba.info()


In [ ]:
# splitting dataframe by groups
# grouping by year
grouped = nba.groupby(nba.year_id)
#get the group of 1947
df_new=grouped.get_group(1947)
df_new

In [ ]:
df_new.count()

*Splitting Pandas Dataframe by sized chunks*

Randon 60%

In [ ]:
# splitting dataframe in a particular size
df_split = nba.sample(frac=.6)
df_split.reset_index()
#df_split
len(df_split)

Split  dataframe in different sets

In [ ]:
#Shuffle the whole dataset first
ds3 = nba.copy()
ds3.sample(frac=1, random_state=42)

In [ ]:
ds3

Split in two dataframes

In [ ]:

list_of_dataframes = np.array_split(ds3, 2)
print("First:")
list_of_dataframes[0]
print('-'*100)
print("Second:")
list_of_dataframes[1]

In [ ]:
# spliting dataframe by row index
# last 1000 rows
df1 = nba.iloc[:1000,:]
# first 1000 rows
df2 = nba.iloc[1000:,:]
print(df1)
print("---------------------------")
print(df2)

*Spliting by Columns (Features)*

In [ ]:
# Split the DataFrame using iloc[] by columns
# first 3 columns
df1 = nba.iloc[:,:3]
# last 3 columns
df2 = nba.iloc[:,3:]
print(df1)
print("---------------------------")
print(df2)

###Join datasets

### Generate a new Dataset file
After the dataset analysis it could be necessary to generate a new dataset.

In [ ]:
import os
#print(os.getcwd())

f=nba.to_csv(path+'newNBA.csv', sep=';', index=False)

### Exporting only a few features

In [ ]:
#or
nba.to_csv(path+'newNbaII.csv',columns=['gameorder','game_id'])

End!